<a href="https://colab.research.google.com/github/megan-the-astronomer/ASTR229/blob/main/thinking_about_statistics_and_noise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Noise is a fact of [the scientific] life

While there are many things we can do to reduce the uncertainty in our measurements, we cannot remove the noise from our data. In astronomical images, noise appears in the form of read noise and/or statistical fluctuations due to the discrete nature of photons.

We can measure and correct for detector effects like the bias but each of these calibration steps adds noise. This is because each calibration frame also has its own associated noise (recall that the read noise is in *every* frame).

For uncorrelated, Gaussian random errors, we propogate uncertainties as follows.
For the sum or difference of two quantities (e.g., subtracing the bias), $z = x - y$, the associated uncertainties $\sigma_x$ and $\sigma_y$ add in quadrature.

\begin{equation}
\sigma_z^2 = \sigma_x^2 + \sigma_y^2
\end{equation}

For multiplicative terms (e.g., dividing by the flat field), $z = x/y$, we sum the fractional uncertainties.   

\begin{equation}
\left( \frac{\sigma_z}{z} \right)^2 = \left( \frac{\sigma_x}{x} \right)^2 + \left( \frac{\sigma_y}{y} \right)^2
\end{equation}

One of the key ways we minimize how much we noise we add to the data when calibrating is by taking multiple frames. This allows us to measure the average value with an uncertainty that decreases with the inverse square root of the number of frames. Mathematically, we are making $N$ measurements of a value (such as the bias) that we'll call $x_i$. Each of those has some associated uncertainty, $\sigma_{x_i}$.

When we calculate the mean, we are computing the sum

\begin{equation}
z = \frac{ \sum_{i}^{N} x_i}{N}
\end{equation}

Using the error propogation equations above, you can show that

\begin{equation}
\sigma_z = \frac{ \sigma_x }{ \sqrt{N} }
\end{equation}

Let's use these equations to look at the uncertainty in our data.

In [ ]:
# start by importing the necessities
import numpy as np
from matplotlib import pyplot as plt

from astropy.io import fits
import astropy.units as u

import glob

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Estimating the read noise from the bias frames

Ideally, in a zero second exposure, the only source of noise is the readnoise. Therefore, we can estimate the readnoise by looking at the standard deviation of the bias frames.

In [ ]:
# get the biases
biases = glob.glob('/content/drive/MyDrive/my_ASTR229_data/biases/*.fits')
biases

['/content/drive/MyDrive/my_ASTR229_data/biases/obj0078.fits',
 '/content/drive/MyDrive/my_ASTR229_data/biases/obj0076.fits',
 '/content/drive/MyDrive/my_ASTR229_data/biases/obj0075.fits',
 '/content/drive/MyDrive/my_ASTR229_data/biases/obj0077.fits',
 '/content/drive/MyDrive/my_ASTR229_data/biases/obj0079.fits',
 '/content/drive/MyDrive/my_ASTR229_data/biases/obj0074.fits',
 '/content/drive/MyDrive/my_ASTR229_data/biases/obj0083.fits',
 '/content/drive/MyDrive/my_ASTR229_data/biases/obj0082.fits',
 '/content/drive/MyDrive/my_ASTR229_data/biases/obj0081.fits',
 '/content/drive/MyDrive/my_ASTR229_data/biases/obj0080.fits',
 '/content/drive/MyDrive/my_ASTR229_data/biases/obj0084.fits',
 '/content/drive/MyDrive/my_ASTR229_data/biases/obj0008.fits',
 '/content/drive/MyDrive/my_ASTR229_data/biases/obj0002.fits',
 '/content/drive/MyDrive/my_ASTR229_data/biases/obj0009.fits',
 '/content/drive/MyDrive/my_ASTR229_data/biases/obj0003.fits',
 '/content/drive/MyDrive/my_ASTR229_data/biases/obj0001

In [ ]:
# read in the images
bias_timestream = [fits.getdata(x) for x in biases]

In [ ]:
# combine, calculate statistics
mean_bias = np.nanmean(bias_timestream, axis=0)
median_bias = np.nanmedian(bias_timestream, axis=0)
stddev_bias = np.std(bias_timestream, axis=0)


In [ ]:
# estimate the read noise
est_noise = np.mean(stddev_bias)
est_noise

np.float64(17.52538063479973)

After averaging the images together, the readnoise should be the only noise in the image. Remember that the uncertainty should be reduced by $1 / \sqrt{N}$ !

In [ ]:
mean_bias_noise = est_noise / np.sqrt(len(biases))
mean_bias_noise

np.float64(3.7364237057440453)

What are the units? We've done the calculation in counts (or ADU - analog to digital units) so to compare with the value recorded in the image header, we need to multiply by the gain.

Look at the header of one of the bias files to see the gain and recorded readnoise.

In [ ]:
bias_header = fits.getheader(biases[0])

In [ ]:
bias_header

SIMPLE  =                    T / Fits standard                                  
BITPIX  =                   16 / Bits per pixel                                 
NAXIS   =                    2 / Number of axes                                 
NAXIS1  =                 1056 / Axis length                                    
NAXIS2  =                 1024 / Axis length                                    
EXTEND  =                    F / File may contain extensions                    
BSCALE  =           1.000000E0 / REAL = TAPE*BSCALE + BZERO                     
BZERO   =           3.276800E4 /                                                
ORIGIN  = 'NOAO-IRAF FITS Image Kernel July 1999' / FITS file originator        
DATE    = '2024-02-15T06:27:22' / Date FITS file was generated                  
IRAF-TLM= '2024-02-15T06:26:29' / Time of last modification                     
OBJECT  = 'Bias    '           / Name of the object observed                    
OBSERVAT= 'MCDONALD         

In [ ]:
gain = bias_header['GAIN']
gain

1.6

Be sure to check the units given in the header!

In [ ]:
gain * mean_bias_noise

np.float64(5.978277929190472)

In [ ]:
bias_header['RDNOISE']

5.87

Remember that the readnoise is a single number because the readout happens at a single place on the CCD, not in each pixel.

Before we move on, let's delete the ```bias_timestream```. Big files take a lot of computer memory. To limit how much data is kept in memory, we can delete variables that contain a lot of data once we're done using them.

In [ ]:
del bias_timestream

# Propogating uncertainties from data calibration

In the same spirit, calculate the uncertainty in the dark and flat field corrections.

To do so, think about what sources of noise contribute to these images. Like every other image, both the dark and flat images have read noise. Both the dark and flats are counting signal and therefore also have Poisson uncertainties. Recall that the Poisson uncertainty goes as $\sqrt{N}$ where $N$ is the number of counts/photons.

Uncertainties add in quadrature, so remember that the total uncertainty in the dark will be the combination of Poisson noise and read noise but reduced by the number of frames.

In [ ]:
# your code to calculate the Poisson uncertainty in the dark frame

In [ ]:
# your code to calculate the total uncertainty in the dark frame

The process for the flat field is substantially similar.

Be careful! When calculating the Poisson noise, be sure to use the combined flat image before you normalize it.

In [ ]:
# your code to calculate the Poisson uncertainty in the flat

We divide the science images by the *normalized* flat. To make the normalized flat, we divided the combined flat field image by its mean so the average value of the flat is 1. This is essentially the equation $z=a \times x$ where $a$ is a constant. In this case, the  uncertainty is $\sigma_z^2 = a^2\sigma_x^2$.

Dividing the flat by a large number to normalize to 1 also reduces the uncertainty by a large amount! Yet another reason to make sure your flat field images have plenty of counts.

In [ ]:
# your code to calculate the total uncertainty in the flat

# Formal propagation of uncertainty in the science image

Now that you have calculated the uncertainty in the calibration frames, propagate that uncertainty through to the science frames. Don't forget to include the uncertainty in the science frame as well!

In [ ]:
# your code to calculate the total uncertainty in the science frame

# Is this a good estimate of the total uncertainty?

Propagating uncertainties in this way gives a good estimate of the formal uncertainty. But remember - there may be other sources of error. Perhaps the most common - and most painful - are systematic errors.

With that in mind, the best way to calculate the uncertainty in your measurement is to measure the spread in your data. How can you do that?

Take, for example, the zero point calculation. You can take one star with a known magnitude and use that to derive the zero point. Try the same thing for a few other stars in the image with known magnitude. What is the spread of those values?

Remember: the most representative uncertainty in your experimental data is the spread of your measurements.

In [ ]:
# your code to calculate the uncertainty in the zeropoint

# So why bother with formal error propagation at all?

Wait - if the best way to estimate the uncertainty of our data is to look at the spread of our measurements, what's the point of all this formal error propagation?

Think about the observing proposal. We had to know something about our source and how we would observe it to estimate the photon count rate and derive an exposure time. We assumed that the dominant source of uncertainty was the Poisson noise from our object. A more robust exposure time calculator does not make this assumption and includes other sources of noise. This is useful for designing observations that might not be limited by just photon noise from the source. Think about directly imaging exoplanets and/or the earliest galaxies.

Understanding the noise and uncertainty we are guaranteed to have in our data can also inform good experimental design. Let's revisit one of the questions we discussed qualitatively in class - is it better to take one long exposure or several short exposures?

### Readnoise

Starting with the readnoise - since it will be present in *every* image - think about how the noise changes depending on the experimental design.

Image you have taken 3 images with identical exposure time and take the **sum**. What is the contribution from the readnoise to this combined image?

In [ ]:
# your code here

What if we instead take 10 images?

In [ ]:
# your code here

Notice that the noise increases as you add more images together.

### Signal

Imagine a light sources that produces one photon per second. Which is better - a single exposure of 100s or 100 exposures of 1s each?

First, determine how much signal you expect in a 100s exposure.

In [ ]:
# your code/answer here

What is the readnoise in the 100s exposure?

In [ ]:
# your code/answer here

What is the signal-to-noise ratio, $S/N$, accounting only for readnoise?

In [ ]:
# your code/answer here

How much **signal** do you expect if you add the 100x1s exposures together?


In [ ]:
# your code/answer here

How much **noise** do you expect if you add the 100x1s exposures together?


In [ ]:
# your code/answer here

What is the $S/N$ in the sum of the 100x1s exposures, accounting only for readnoise?

In [ ]:
# your code/answer here

Considering *only* the readnoise, which is better? One long exposure or many short exposures?

In [ ]:
# your code/answer here

### A more realistic estimate including Poisson noise

Recall that the Poisson noise is $\sigma = \sqrt{N}$.

What is the expected Poisson noise in the 100s exposure?

In [ ]:
# your code/answer here

What is the expected Poisson noise in each of the 1s exposures?

In [ ]:
# your code/answer here

When you sum the 100x1s images, what is the expected Poisson noise?

In [ ]:
# your code/answer here

Including Poisson noise, which is better? A single long exposure or many short exposures?

In [ ]:
# your code/answer here

For the interested reader, try running the above calculations again for a source that emits 100 photons per second.

As always, the best experiemental design depends on your source and what you are trying to measure.